In [1]:
import numpy as np
import os
import cv2
import pandas as pd
import time

from keras.optimizers import SGD, Adagrad, Adadelta, RMSprop, Adam, Nadam
import keras.backend as K
import tensorflow as tf
import scipy.io as sio
from keras.models import Model
from keras.regularizers import l2, l1
from keras.models import Sequential
from keras.layers import Input, Concatenate, LeakyReLU, UpSampling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import RMSprop, Adam, Adadelta
from keras.callbacks import Callback, ModelCheckpoint

Using TensorFlow backend.


In [ ]:
#---------------------------
#  Input Maps Folders
#---------------------------
training_labels_path = 'E:/training_filenames.txt'
test_labels_path = 'E:/testing_filenames.txt'
pool5_features = 'E:/pool5_feats'
gap_features = 'E:/GAP_feats'
product_maps = 'E:/product_maps'

#------------------
#  Outputs
#------------------
final_results_path = 'E:/results.txt'    
weights_savepath = 'E:/FusionNet/weights/'


In [2]:
#---------------------------------------------------------------------
#  Data Generator function to generate batches according to batch size
#---------------------------------------------------------------------
def generator(train_indices, batch_size, itr, lines, image_size=336):
    batch_images = []
    batch_labels = []
    batch_pmap = []
    batch_product = []
    startInd = itr*batch_size
    endInd = startInd + batch_size
    p_mapsize = 32
    fmap_size = 42
    
    for i in range(startInd, endInd):
        line = lines[i].split(' ')
        
        #pool5 features
        image_path = os.path.join(pool5_features, line[0] + '.mat')
        fmap1 = sio.loadmat(image_path)
        fmap1 = fmap1['fmap']
        fmap = fmap1.ravel()
        
        #GAP features
        image_path1 = os.path.join(gap_features, line[0] + '.mat')
        fmap11 = sio.loadmat(image_path1)
        fmap11 = fmap11['Pgap']
        fmap1 = fmap11.ravel()
        
        #product maps
        image_path2 = os.path.join(product_maps, line[0] + '.mat')
        fmap22 = sio.loadmat(image_path2)
        fmap22 = fmap22['product_pmap']
        fmap2 = np.zeros((fmap_size,fmap_size,1024))
        fmap2[:,:,:] = fmap22[:,:,:]
        
        batch_images.append(fmap)
        batch_pmap.append(fmap1)
        batch_product.append(fmap2)
        batch_labels.append(np.int(line[1]))
        
    batch_images = np.array(batch_images, dtype=np.float32)
    batch_pmap = np.array(batch_pmap, dtype=np.float32)
    batch_product = np.array(batch_product, dtype=np.float32)
    batch_labels = np.array(batch_labels, dtype=np.float32)
    
    
    return batch_images, batch_pmap, batch_product, batch_labels

In [3]:
#------------------------
# RMSE loss function
#------------------------
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

In [4]:
#--------------
#  SSNet Model
#--------------
def create_model():
    
    f_map = Input(shape=(1024,), name='DenseNet')
    p_map = Input(shape=(1024,), name='SSNEt')
    prod_map = Input(shape=(42,42,1024), name='product')
    
    x_d = Dense(512, init='glorot_normal')(f_map)
    x_d = LeakyReLU(alpha=0.3)(x_d)
    x_d = Dropout(0.5)(x_d)
    
    x_s = Dense(512, init='glorot_normal')(p_map)
    x_s = LeakyReLU(alpha=0.3)(x_s)
    x_s = Dropout(0.5)(x_s)
    
    x_p = Convolution2D(1, (1, 1), init='glorot_normal', W_regularizer=l1(0.001))(prod_map)
    x_p = LeakyReLU(alpha=0.3)(x_p)
    x_p = Flatten()(x_p)
    x_p = Dense(512, init='glorot_normal')(x_p)
    x_p = LeakyReLU(alpha=0.3)(x_p)
    x_p = Dropout(0.5)(x_p)
    
    
    x = Concatenate(axis=-1)([x_d,  x_s, x_p]) 
    x = Dense(512, init='glorot_normal')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Dropout(0.5)(x)
    x = Dense(32, init='glorot_normal')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Dropout(0.5)(x)
    x = Dense(1, init='glorot_normal')(x)    
    x = Activation('linear')(x)
    
    adagrad=Adagrad(lr=0.01, epsilon=1e-08)
    
    model_finetune = Model(inputs=[f_map, p_map, prod_map], outputs=x)    
    model_finetune.compile(loss=root_mean_squared_error, optimizer=adagrad, metrics=['accuracy'])
    return model_finetune

In [3]:
model = create_model()
model.summary()

In [1]:
#------------------------------
#   Model Training
#------------------------------


#Model Parameters
batch_size = 1
nb_epoch = 3000

model = create_model()
lines = [line.rstrip('\n') for line in open(training_labels_path)]

indices = np.arange(len(lines))
train_indices = indices[:]
print(train_indices.shape)
iterations_t = np.int(len(train_indices)/batch_size)
t_loss_ep = []

#loop for number of epochs
for epoch in range(nb_epoch):
    
    t_loss = []
    start = time.time()
    
    for i in range(iterations_t):
        X_train, X_pmap, X_prod, y_train = generator(train_indices, batch_size, i, lines, image_size=336)  
        history = model.fit([X_train, X_pmap, X_prod], y_train, verbose=0)
        t_loss.append(history.history['loss'])
     
    stop = time.time()
    duration = stop - start
    t_loss_ep.append(np.mean(np.array(t_loss, dtype=np.float32)))
    print("Epoch: {}  Training Loss: {}  Time: {} sec".format(epoch+209, t_loss_ep[epoch], duration) )
    
    #save weigths for each epoch      
    model_name = weights_savepath + str(epoch) + "_Epochs.hdf5"
    model.save_weights(model_name)
    

In [2]:
#------------------------------
#   Model Testing
#------------------------------

lines = [line.rstrip('\n') for line in open(test_labels_path)]

#Load weigth. Please replace XXX with the epoch number upto which the model was trained
model.load_weights(weights_savepath + str(XXX) + '_Epochs.hdf5')

data = ""
data_low = ""
data_low_gt = ""
data_high = ""
data_high_gt = ""
data_medium = ""
data_medium_gt = ""

mean_abs_error = 0
mean_sqrd_error = 0

mean_abs_error_low = 0
mean_sqrd_error_low = 0

mean_abs_error_high = 0
mean_sqrd_error_high = 0

mean_abs_error_medium = 0
mean_sqrd_error_medium = 0

total_data = len(lines)
count = 0
count_low = 0
count_high = 0
count_medium = 0
fmap_size = 42
p_mapsize = 32

print(len(lines))
for i in range(len(lines)):
    line = lines[i].split(' ')

    image_path = os.path.join('E:/ITU/Dr_Mohsen/houseCounting/new_data_fmaps/pool5', line[0] + '.mat')
    fmap1 = sio.loadmat(image_path)
    fmap1 = fmap1['fmap']
    fmap = fmap1.ravel()
    test_image = np.zeros((1,1024))
    test_image[0,:] = fmap

    image_path1 = os.path.join('E:/ITU/Dr_Mohsen/houseCounting/new_data_fmaps/pgap_maps', line[0] + '.mat')
    fmap11 = sio.loadmat(image_path1)
    fmap11 = fmap11['Pgap']
    fmap1 = fmap11.ravel()
    test_pmap = np.zeros((1,1024))
    test_pmap[0, :] = fmap1
    
    image_path2 = os.path.join('E:/ITU/Dr_Mohsen/houseCounting/new_data_fmaps/pf_maps', line[0] + '.mat')
    fmap22 = sio.loadmat(image_path2)
    fmap22 = fmap22['pfmap']
    fmap2 = np.zeros((fmap_size,fmap_size,1024))
    fmap2[:,:,:] = fmap22[:,:,:]
    test_product = np.zeros((1,42,42,1024))
    test_product[0, :, :, :] = fmap2
         
    
    output = model.predict([test_image, test_pmap, test_product])
    if output < 0:
        output = 0
        
#     print(line[0], ' ', output)
    print([gt_count, output])
    
    
    #calculating in various count ranges as described in paper
    abs_error = np.abs(gt_count - output)
    mean_abs_error = mean_abs_error + abs_error
    
    sqrd_error = (gt_count - output) * (gt_count - output)
    mean_sqrd_error = mean_sqrd_error + sqrd_error  
    
    if gt_count >= 0 and gt_count <= 30:
            count_low = count_low + 1
            mean_abs_error_low = mean_abs_error_low + abs_error
            mean_sqrd_error_low = mean_sqrd_error_low + sqrd_error
            data_low = data_low + line[0] + " " + str(float(output)) + "\n"
            data_low_gt = data_low_gt + line[0] + " " + str(float(gt_count)) + "\n"
        
    if gt_count >= 61 and gt_count <= 100:
            count_high = count_high + 1
            mean_abs_error_high = mean_abs_error_high + abs_error
            mean_sqrd_error_high = mean_sqrd_error_high + sqrd_error
            data_high = data_high + line[0] + " " + str(float(output)) + "\n"
            data_high_gt = data_high_gt + line[0] + " " + str(float(gt_count)) + "\n"
            
    if gt_count >= 31 and gt_count <= 60:
            count_medium = count_medium + 1
            mean_abs_error_medium = mean_abs_error_medium + abs_error
            mean_sqrd_error_medium = mean_sqrd_error_medium + sqrd_error
            data_medium = data_medium + line[0] + " " + str(float(output)) + "\n"
            data_medium_gt = data_medium_gt + line[0] + " " + str(float(gt_count)) + "\n"
    
    
    data = data + line[0] + " " + str(float(output)) + "\n"

#write final results into a text file
f = open(final_results_path,'w')
f.write(data)

f_mean_abs_error = mean_abs_error / total_data
f_mean_sqrd_error = mean_sqrd_error / total_data

f_mean_abs_error_low = mean_abs_error_low / count_low
f_mean_sqrd_error_low = mean_sqrd_error_low / count_low

f_mean_abs_error_high = mean_abs_error_high / count_high
f_mean_sqrd_error_high = mean_sqrd_error_high / count_high

f_mean_abs_error_medium = mean_abs_error_medium / count_medium
f_mean_sqrd_error_medium = mean_sqrd_error_medium / count_medium

print('##################################### ERROR VALUES ###########################################')
print("Mean Squared Error: {} ".format(f_mean_sqrd_error))
print("Mean Absolute Error: {} ".format(f_mean_abs_error))
print("Number of Instances: {} ".format(total_data))
print('##############################################################################################')

print('##################################### ERROR VALUES-LOW ###########################################')
print("Mean Squared Error: {} ".format(f_mean_sqrd_error_low))
print("Mean Absolute Error: {} ".format(f_mean_abs_error_low))
print("Number of Instances: {} ".format(count_low))
print('##############################################################################################')

print('##################################### ERROR VALUES-MEDIUM ###########################################')
print("Mean Squared Error: {} ".format(f_mean_sqrd_error_medium))
print("Mean Absolute Error: {} ".format(f_mean_abs_error_medium))
print("Number of Instances: {} ".format(count_medium))
print('##############################################################################################')

print('##################################### ERROR VALUES-HIGH ###########################################')
print("Mean Squared Error: {} ".format(f_mean_sqrd_error_high))
print("Mean Absolute Error: {} ".format(f_mean_abs_error_high))
print("Number of Instances: {} ".format(count_high))
print('##############################################################################################')